# Cluster + RF code for last-man-standing hackathon by AnalyticsVidhya
### Code created on : 31st Jan 2015 | Author: Bargava

Link to competition
http://datahack.analyticsvidhya.com/contest/last-man-standing

#### Solution Approach
In this, we will create clusters from the training dataset. We will compute the distances of each training point to those cluster centers. That will be used as input to the `xgboost` model for training. 

In [1]:
#Import the necessary libraries
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
from sklearn.kernel_ridge import KernelRidge
from sklearn.svm import SVR
from sklearn.cluster import KMeans

In [2]:
#read train, test and samplesub datasets
train = pd.read_csv("../data/trainUpdated.csv")
test = pd.read_csv("../data/testUpdated.csv")
samplesub = pd.read_csv("../data/samplesubUpdated.csv")
label = pd.read_csv("../data/labelsUpdated.csv")

### k-means clustering

In [4]:
#set the parameters
kmeans_estimator = KMeans(n_clusters=300, n_init=1, n_jobs=-2)

In [5]:
#fit kmeans
kmeans_estimator.fit(train)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=300, n_init=1,
    n_jobs=-2, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [6]:
#transform data to cluster-distance space - both train and test
new_train = kmeans_estimator.transform(train)
new_test = kmeans_estimator.transform(test)

In [7]:
print new_train.shape, new_test.shape

(88858, 300) (59310, 300)


In [10]:
train = np.array(train)
test = np.array(test)

In [15]:
train = np.concatenate((train, new_train), axis=1)
test = np.concatenate((test, new_test), axis=1)

In [16]:
print train.shape, test.shape

(88858, 312) (59310, 312)


### `xgboost` model

In [17]:
params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.5
params["colsample_bytree"] = 0.5
params["silent"] = 0
params["max_depth"] = 14
params["nthread"] = 6
params["gamma"] = 3
params["objective"] = "multi:softmax"
params["num_class"] = 3
params["verbose"] = 2
params["eta"] = 0.3
params["base_score"] = 0
params["eval_metric"] = "merror"
params["seed"] = 13

plst = list(params.items())
num_rounds = 400

In [18]:
xgtrain_pv = xgb.DMatrix(np.array(new_train).astype(float), label=np.array(label).astype(float))
watchlist = [(xgtrain_pv, 'train')]

In [19]:
model_1_xgboost = xgb.train(plst, xgtrain_pv, num_rounds)

In [29]:
model_1_xgboost.save_model("../model/model1xgboost.model")

In [20]:
#Predicting the model
model_1_predict = model_1_xgboost.predict(xgb.DMatrix(np.array(new_test).astype(float)))

In [21]:
samplesub.columns = ["ID"]

In [22]:
samplesub["Crop_Damage"] = model_1_predict

In [23]:
#Write the prediction to a csv
samplesub.to_csv("../submission/submission_31jan_1.csv", index=False)

In [56]:
#This coded 0.834853657172 on LB